## Assignment 1
### Jiawei Huang

### Implement the Boyer-Moore algorithm

### Chapter 1, exercise 11
>Let T be a text string of length m and let S be a multiset of n characters. The problem is to find all substrings in T of  length n that are formed by the characters of S. For example, let S = (a, a, b, c} and T = abahgcabah. Then caba is a substring of T formed from the characters of S. Give a solution to this problem that runs in O(m) time. The method should also be able to state, for each position i, the length of  the longest substring in T starting at i that can be formed from S.

__Answer:__

Method based on sliding window. Creating a map containing the letters in multiset $S$ with frequency counts of the letters be the value, call it __target__.

Start by processing the first n letters in text. For each such letter if it appears as a key in the target dictionary decrease the corresponding value by 1. The goal is to drive all target values to 0. Define __discrepancy__ to be the sum of the absolute values of the values after processing the first window of m letters. Then move the n-length window 1 space to the right, check the letter preceeding the left window side(this letter just moves out of the window) and the letter at the right end of the window(just be coverd by the window) and maintain __target__ and __discrepancy__.

__annotation:__
![](chapter1_11.png)
_Fig.1: sliding window_

__pseudocode:__

__input: s1: pattern, s2: text__ 

__output: list of start positions of substrings in s2, which are formed by letters in s1__
```
m <- length of s1, n <- length of s2
if m > n
    return no matching

occurrence <- {c:n} # c is each character in s1, n is the difference between the first m characters in s2 and s1
discrepancy <- sum(n for c:n in occurrence)
match = []
for i <- 2 to n - m + 1
    if discrepancy==0
         add i to match
    if s2(i-1) in occurrence.keys
        occurence[s2(i-1)] --
    if s2(i + m - 1) in occurrence.keys
        occurence[s2(i+m-1)] ++
return match

```
```python
# s1 is pattern S, s2 is text
def match(s1,s2):
    m = len(s1)
    n = len(s2)
    if m > n: return false
    target = dict.fromkeys(s1,0)
    for c in s1: target[c] += 1

    #process initial window
    for i in range(m):
        c = s2[i]
        if c in target:
            target[c] -= 1
    discrepancy = sum(abs(target[c]) for c in target)

    #repeatedly check then slide:
    for i in range(m,n):
        if discrepancy == 0:
            return True
        else:
            #first process letter from m steps ago from s2
            c = s2[i-m]
            if c in target:
                target[c] += 1
                if target[c] > 0: #just made things worse
                    discrepancy +=1
                else:
                    discrepancy -=1
            #now process new letter:
            c = s2[i]
            if c in target:
                target[c] -= 1
                if target[c] < 0: #just made things worse
                    discrepancy += 1
                else:
                    discrepancy -=1
    #if you get to this stage:
    return discrepancy == 0
```

<!-- 
```
match(T, S):

1: n <- length(T), m<- length(S)
2: if m > n return false
3: target <- dict.fromkeys(S, 0)
4: for i <- 1 to m do
5:     target( S(i) ) += 1
6: 
7: for i <- 1 to m do
7:     if T(i) in target.keys do
8:         target(i) -= 1
9: discrepancy <= sum( abs( target.values ) )
10:
11: for i <- m+1 to n do
12:     if discrepancy == 0 
13:         return true
14:     else
15:         if T(i-m) in target.keys do
```
 -->

Since there are no nested loop and each pass through the main loop involves just a few dictionary lookups, comparisons, addition and subtractions, the overall algorithm is linear.

For each position $i$, the length of the longest substring in $T$ starting at $i$ that can be formed from $S$ can be calculated, from position $i$, length increses until come to a letter not in $S$.  

### Chapter 6, exercise 1
>Construct an infinite  family of strings  over a  fixed alphabet, where the total length of the edge-labels on their suffix trees grows faster than O(m) (m is the length of the string). That is, show that linear-time suffix tree algorithms would be impossible if edge-labels were written explicitly on the edges.

__Answer:__

Consider such string $abaabbaaabbb...(a)_i(b)_i...$ where $(a)_i$ means repeat i times,in this way we only have alphabet size of 2, but the edge labels on the suffix tree grows faster than $\theta(m)$. 
![image.png](infinite_edges.png)

The total number of character of the suffix tree is:
$$\sum_{1}^{m} = \frac{m(m+1)}{2}\sim\theta(m^2)$$

If edge-labels were written explicitly on the edges, the time for the algorithm is at least as large as the size of the output, which is $\theta(m^2)$, so It cannot be linear-time.

### Chapter 7, exercise 1
>Given a set S of k strings, we want to find every string in S that is a substring of  some other string in S. Assuming that the total length of  all the strings is n, give an O(n)-time algorithm to solve this problem. This result will be needed in algorithms for the shortest superstring problem.

__Answer:__
Using Generalized Sufix Tree. 

+ Construct a suffix tree for the single string $T_1\$_1T_2\$_2...T_k\$_k$ ($O(n)$) 
+ Run a DFS over the suffix tree and prune the invalid suffixes, namely, get the generalized suffix tree. Leaves are tagged with $i:j$, meaning “$jth$ suffix of string $T_i$”. When doing DFS, annotate the original strings(tagged $i:0$). ($O(n)$)
+ Run a DFS over the selected branches and check whether there is a node on the branch whose out edge is also the original string in $S$ and has a end $\$$ sign only. Get the string $s$ and its sibling strings which have more than one letters on its branch. This takes $O(n)$.

Overall complexity is $O(n)$

Following figure shows the geneal idea of the algorithm. First we select branches which is tagged $i:0$, $i = 1, 2...,k$. Then we go through these edges and try to find nodes whose out edge contains only end string dollar sign. Here we find string $j$ is a substring of string $i$. 
![image.png](GST.png)

### Chapter 7, exercise 2
>For a string S of length n, show how to compute the N(i), L(i), L'(i) and Spi values (discussed in Sections 2.2.4 and 2.3.2) in O(n) time directly from a suffix tree for S.

__Answer:__

$N(i):$  $N(i)$ is the length of the longest suffix of substring $S[1..i]$ which is also a suffix of full string $S$. 
So first reverse string $S$ and get $S^r$ and then construct suffix tree based on $S^r$. And then find internal nodes on branch $S^r[1..n]$. $N(i)$ is the number of common elements between $S^r[1..n]$ and sub-branch whose length is i. If there is no sub-branch with length i, then $N(i) = 0$
![image.png](reversed_ST.png)

$L(i):$ $L(i)$ is the largest position less than $n$ such that $S[i..n]$ matches a suffix of $S[1..L(i)]$ 
Construct suffix tree of $S$, and start with $S[i]$ branch, if there is a node at a distance $n-i+1$ from root on this branch. Then choose next node on this branch and calculate the distance $l$ between the two nodes, if there is not an extra node on this branch, make $l$ be the distance from the node to end. Then $L(i) = n-l$, if there is not a node at $n-i+1$, then $L(i) = 0$. For example, we want to calculate $L(8)$ of following string $S=cabdabdab$. Here $S(8) = a, n = 9$, we first look at branch start with $a$, and we find there is a branch of $ab\$$, which means $L(8)$ is not 0, and we find next node and the distance between the two nodes $l=3$, so $L(8)=n-l=9-3=6$
![image.png](L_array.png)

$L'(i):$ $L'(i)$ is the largest position less than $n$ such that $S[i..n]$ matches a suffix of $S[1..L(i)]$ and the character preceding that suffix is not equal to $S[i-1]$.
Similar to the calculation of $L(i)$, first start with $S[i]$ branch and see whether there is a branch at $n-i+1$, if no extra branch, set $L'(i)=0$, else find distance between next node or end to this node, but this time we should compare the character preceding the suffix with $S[i-1]$, if they are equal, ignore this node and go to next node, if there is not an extra node, set $L'(i)=0$

$SP(i):$ $SP(i)$ is the length of the longest suffix of $S[1..i]$ matching a prefix of $S$. We can first get $Z$ values and then convert them into $SP$ values. 

$Z$ value is the distance from root to node on the original string, that is to say $Z(n-l-d + 1) = d$, $n$ is the length of original string, l is the number of letters out of the nodes, d is the distance from root to the node. For example, calculate the $Z$ values of string $AAAGCAGTCA$. We can get the suffix tree like below. And we found 2 nodes $L$, $M$ on the original string, and 3 branches out of $L$, 2 branches out of $M$. For node $L$, distance between root $R$ to it is 1, there is one $\$$ branch out of $L$, which means $l = 0$, so $Z(10 - 0 - 1 + 1) = Z(10) = 1$, the second branch out of $L$ is $GTCA\$$, so $l = 4$, $Z(10 - 4 -1 + 1) = Z(6) = 1$, the third branch out of $L$ is $GCAGTCA\$\$$, so $l = 7$, $Z(10 - 7 - 1 + 1) = Z(3) = 1$, and similarly we can get $Z(2) = 2$. 
![image.png](Z_values.png)

After getting $Z$ values, we can then convert them into SP values. Here is the pseudocode.

  __input: Z array  
  output: sp__  
  ```
  for i <- 1 to n:  
      sp'(i) <- 0  
  for j <- n downto 2:  
      i <- j + Z(j) - 1  
      sp'(i) <- Z(j)  
  sp = sp'  
  for i <- n-1 downto 2:  
      sp(i) <- max(sp(i+1) - 1, sp'(i))  
  return sp
  ```